In [1]:
import pandas as pd
import numpy as np
import time


SCRATCH_DIR = "/scratch/siads696f23_class_root/siads696f23_class/psollars"
SCRATCH_DIR = "../data"

start_time = time.time()
one_year_df = pd.read_parquet(f"{SCRATCH_DIR}/all_features_2019.parquet")
end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

Elapsed time: 53.0813 seconds


In [2]:
start_time = time.time()

# Remove cancelled and diverted flights
df = one_year_df[one_year_df["Cancelled"].eq(0) & one_year_df["Diverted"].eq(0)].copy()

print(f"Removed {len(one_year_df) - len(df)} rows")

# We'll need to recompute the delayed label
# If the flight is delayed on departure or arrival, consider it delayed overall

df["delayed"] = (df["DepDel15"].eq(1)) | (df["ArrDel15"].eq(1))
end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

df.head()

Removed 155683 rows
Elapsed time: 70.2201 seconds


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityMarketID,...,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Origin_LATITUDE,Origin_LONGITUDE,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_LATITUDE,Dest_LONGITUDE,delayed
24733033,2019,1,1,4,5,2019-01-04 00:00:00,OO,945SW,5657,34236,...,2023.0,San Luis County Regional,"San Luis Obispo, CA",35.237222,-120.642500,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24733034,2019,1,1,4,5,2019-01-04 00:00:00,OO,932SW,5658,31453,...,2023.0,George Bush Intercontinental/Houston,"Houston, TX",29.984444,-95.341389,Northwest Arkansas National,"Fayetteville, AR",36.281667,-94.307778,False
24733035,2019,1,1,4,5,2019-01-04 00:00:00,OO,932SW,5658,34783,...,2023.0,Springfield-Branson National,"Springfield, MO",37.245556,-93.388611,George Bush Intercontinental/Houston,"Houston, TX",29.984444,-95.341389,False
24733036,2019,1,1,4,5,2019-01-04 00:00:00,OO,916SW,5659,32389,...,2023.0,Sloulin Field International,"Williston, ND",48.178056,-103.642222,Denver International,"Denver, CO",39.861667,-104.673056,False
24733037,2019,1,1,4,5,2019-01-04 00:00:00,OO,107SY,5660,33342,...,2023.0,General Mitchell International,"Milwaukee, WI",42.946944,-87.896944,Denver International,"Denver, CO",39.861667,-104.673056,False


In [3]:
pd.concat([df["Origin"], df["Dest"]]).value_counts().head(10).index

Index(['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO', 'LGA'], dtype='object')

In [6]:
df.value_counts("Dest").head(20).index

Index(['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO', 'LGA',
       'LAS', 'DTW', 'MSP', 'BOS', 'SEA', 'MCO', 'DCA', 'EWR', 'AUS', 'JFK'],
      dtype='object', name='Dest')

In [7]:
# Filter out for only our top airlines

df.value_counts("Reporting_Airline").head(10).index

# WN    Southwest Airlines Co.
# DL    Delta Air Lines, Inc.
# AA    American Airlines Inc.
# OO    SkyWest Airlines
# UA    United Airlines, Inc.
# YX    Republic Airways Inc
# MQ    Envoy Air Inc.
# B6    Jetblue Airways Corporation
# OH    PSA Airlines, Inc.
# AS    Alaska Airlines Inc.

Index(['WN', 'DL', 'AA', 'OO', 'UA', 'YX', 'MQ', 'B6', 'OH', 'AS'], dtype='object', name='Reporting_Airline')

In [8]:
df_top_airlines = df[df["Reporting_Airline"].isin(["WN", "DL", "AA", "OO", "UA"])]

df_top_airlines

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityMarketID,...,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Origin_LATITUDE,Origin_LONGITUDE,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_LATITUDE,Dest_LONGITUDE,delayed
24733033,2019,1,1,4,5,2019-01-04 00:00:00,OO,945SW,5657,34236,...,2023.0,San Luis County Regional,"San Luis Obispo, CA",35.237222,-120.642500,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24733034,2019,1,1,4,5,2019-01-04 00:00:00,OO,932SW,5658,31453,...,2023.0,George Bush Intercontinental/Houston,"Houston, TX",29.984444,-95.341389,Northwest Arkansas National,"Fayetteville, AR",36.281667,-94.307778,False
24733035,2019,1,1,4,5,2019-01-04 00:00:00,OO,932SW,5658,34783,...,2023.0,Springfield-Branson National,"Springfield, MO",37.245556,-93.388611,George Bush Intercontinental/Houston,"Houston, TX",29.984444,-95.341389,False
24733036,2019,1,1,4,5,2019-01-04 00:00:00,OO,916SW,5659,32389,...,2023.0,Sloulin Field International,"Williston, ND",48.178056,-103.642222,Denver International,"Denver, CO",39.861667,-104.673056,False
24733037,2019,1,1,4,5,2019-01-04 00:00:00,OO,107SY,5660,33342,...,2023.0,General Mitchell International,"Milwaukee, WI",42.946944,-87.896944,Denver International,"Denver, CO",39.861667,-104.673056,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32288695,2019,4,12,1,7,2019-12-01 00:00:00,DL,346DN,830,30397,...,2023.0,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,LaGuardia,"New York, NY",40.777222,-73.872500,False
32288696,2019,4,12,1,7,2019-12-01 00:00:00,DL,697DL,831,31454,...,2023.0,Orlando International,"Orlando, FL",28.429444,-81.308889,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,True
32288697,2019,4,12,1,7,2019-12-01 00:00:00,DL,697DL,831,31650,...,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,Orlando International,"Orlando, FL",28.429444,-81.308889,True
32288698,2019,4,12,1,7,2019-12-01 00:00:00,DL,6714Q,832,30852,...,2023.0,Ronald Reagan Washington National,"Washington, DC",38.851389,-77.037778,Salt Lake City International,"Salt Lake City, UT",40.788333,-111.977778,False


In [15]:
df_top_airline_airport = df[
    df["Reporting_Airline"].isin(["WN", "DL", "AA", "OO", "UA"])
    & (
        df["Dest"].isin(
            ["ATL", "ORD", "DEN", "DFW", "LAX", "PHX", "SFO", "MSP", "DTW", "LAS"]
        )
        | df["Origin"].isin(
            ["ATL", "ORD", "DEN", "DFW", "LAX", "PHX", "SFO", "MSP", "DTW", "LAS"]
        )
    )
]
# 1621989 rows × 109 columns
df_top_airline_airport

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityMarketID,...,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Origin_LATITUDE,Origin_LONGITUDE,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_LATITUDE,Dest_LONGITUDE,delayed
24733033,2019,1,1,4,5,2019-01-04 00:00:00,OO,945SW,5657,34236,...,2023.0,San Luis County Regional,"San Luis Obispo, CA",35.237222,-120.642500,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24733036,2019,1,1,4,5,2019-01-04 00:00:00,OO,916SW,5659,32389,...,2023.0,Sloulin Field International,"Williston, ND",48.178056,-103.642222,Denver International,"Denver, CO",39.861667,-104.673056,False
24733037,2019,1,1,4,5,2019-01-04 00:00:00,OO,107SY,5660,33342,...,2023.0,General Mitchell International,"Milwaukee, WI",42.946944,-87.896944,Denver International,"Denver, CO",39.861667,-104.673056,False
24733038,2019,1,1,4,5,2019-01-04 00:00:00,OO,114SY,5661,30559,...,2023.0,Seattle/Tacoma International,"Seattle, WA",47.450000,-122.311667,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24733039,2019,1,1,4,5,2019-01-04 00:00:00,OO,945SW,5664,32457,...,2023.0,San Francisco International,"San Francisco, CA",37.618889,-122.375556,Boise Air Terminal,"Boise, ID",43.564444,-116.222778,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32288693,2019,4,12,1,7,2019-12-01 00:00:00,DL,865DA,828,31650,...,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,True
32288694,2019,4,12,1,7,2019-12-01 00:00:00,DL,844DN,829,31650,...,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,Salt Lake City International,"Salt Lake City, UT",40.788333,-111.977778,False
32288695,2019,4,12,1,7,2019-12-01 00:00:00,DL,346DN,830,30397,...,2023.0,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,LaGuardia,"New York, NY",40.777222,-73.872500,False
32288696,2019,4,12,1,7,2019-12-01 00:00:00,DL,697DL,831,31454,...,2023.0,Orlando International,"Orlando, FL",28.429444,-81.308889,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,True


In [10]:
df_top_airlines.value_counts("Dest").head(20).index

# Index(['ATL', 'ORD', 'DEN', 'DFW', 'LAX', 'PHX', 'SFO', 'MSP', 'DTW', 'LAS',
#    'CLT', 'SLC', 'AUS', 'MCO', 'IAH', 'BWI', 'LGA', 'MDW', 'SAN', 'EWR'],


pd.concat([df_top_airlines["Origin"], df_top_airlines["Dest"]]).value_counts().head(
    20
).index

Index(['ATL', 'ORD', 'DEN', 'DFW', 'LAX', 'PHX', 'SFO', 'MSP', 'DTW', 'LAS',
       'CLT', 'SLC', 'AUS', 'MCO', 'IAH', 'BWI', 'LGA', 'MDW', 'EWR', 'SAN'],
      dtype='object')

In [11]:
df_top_airlines = df[df["Reporting_Airline"].isin(["WN", "DL", "AA", "B6", "UA", "AS"])]

df_top_airlines

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityMarketID,...,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Origin_LATITUDE,Origin_LONGITUDE,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_LATITUDE,Dest_LONGITUDE,delayed
24798424,2019,1,1,11,5,2019-01-11 00:00:00,AS,364VA,1592,34262,...,2022.0,Palm Springs International,"Palm Springs, CA",33.829722,-116.506667,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24798426,2019,1,1,11,5,2019-01-11 00:00:00,AS,628VA,1595,34262,...,2023.0,Palm Springs International,"Palm Springs, CA",33.829722,-116.506667,San Francisco International,"San Francisco, CA",37.618889,-122.375556,True
24798427,2019,1,1,11,5,2019-01-11 00:00:00,AS,626VA,1603,33570,...,2022.0,San Diego International,"San Diego, CA",32.733611,-117.189722,Portland International,"Portland, OR",45.588611,-122.596944,True
24798428,2019,1,1,11,5,2019-01-11 00:00:00,AS,844VA,1621,30423,...,2023.0,Austin - Bergstrom International,"Austin, TX",30.194444,-97.670000,Seattle/Tacoma International,"Seattle, WA",47.450000,-122.311667,False
24798429,2019,1,1,11,5,2019-01-11 00:00:00,AS,844VA,1621,30423,...,2023.0,Robert Mueller Municipal,"Austin, TX",30.298056,-97.701389,Seattle/Tacoma International,"Seattle, WA",47.450000,-122.311667,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32288695,2019,4,12,1,7,2019-12-01 00:00:00,DL,346DN,830,30397,...,2023.0,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,LaGuardia,"New York, NY",40.777222,-73.872500,False
32288696,2019,4,12,1,7,2019-12-01 00:00:00,DL,697DL,831,31454,...,2023.0,Orlando International,"Orlando, FL",28.429444,-81.308889,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,True
32288697,2019,4,12,1,7,2019-12-01 00:00:00,DL,697DL,831,31650,...,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,Orlando International,"Orlando, FL",28.429444,-81.308889,True
32288698,2019,4,12,1,7,2019-12-01 00:00:00,DL,6714Q,832,30852,...,2023.0,Ronald Reagan Washington National,"Washington, DC",38.851389,-77.037778,Salt Lake City International,"Salt Lake City, UT",40.788333,-111.977778,False


In [12]:
print(df_top_airlines.value_counts("Dest").head(20).index)

# Index(['ATL', 'ORD', 'DEN', 'DFW', 'LAX', 'PHX', 'SFO', 'MSP', 'DTW', 'LAS',
#    'CLT', 'SLC', 'AUS', 'MCO', 'IAH', 'BWI', 'LGA', 'MDW', 'SAN', 'EWR'],


pd.concat([df_top_airlines["Origin"], df_top_airlines["Dest"]]).value_counts().head(
    20
).index

Index(['ATL', 'LAX', 'DFW', 'DEN', 'ORD', 'PHX', 'SEA', 'SFO', 'LAS', 'BOS',
       'CLT', 'MCO', 'AUS', 'JFK', 'MSP', 'EWR', 'BWI', 'SAN', 'MDW', 'LGA'],
      dtype='object', name='Dest')


Index(['ATL', 'LAX', 'DFW', 'DEN', 'ORD', 'PHX', 'SEA', 'SFO', 'LAS', 'BOS',
       'CLT', 'MCO', 'AUS', 'JFK', 'MSP', 'EWR', 'BWI', 'SAN', 'MDW', 'LGA'],
      dtype='object')

In [20]:
df_top_airline_airport = df[
    df["Reporting_Airline"].isin(["WN", "DL", "AA", "B6", "UA", "AS"])
    & (
        df["Dest"].isin(
            ["ATL", "LAX", "DFW", "DEN", "ORD", "PHX", "SEA", "SFO", "LAS", "BOS"]
        )
        | df["Origin"].isin(
            ["ATL", "LAX", "DFW", "DEN", "ORD", "PHX", "SEA", "SFO", "LAS", "BOS"]
        )
    )
]
# 2769026 rows × 109 columns
df_top_airline_airport

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginCityMarketID,...,faa_registration_origin,Origin_DISPLAY_AIRPORT_NAME,Origin_DISPLAY_AIRPORT_CITY_NAME_FULL,Origin_LATITUDE,Origin_LONGITUDE,Dest_DISPLAY_AIRPORT_NAME,Dest_DISPLAY_AIRPORT_CITY_NAME_FULL,Dest_LATITUDE,Dest_LONGITUDE,delayed
24798424,2019,1,1,11,5,2019-01-11 00:00:00,AS,364VA,1592,34262,...,2022.0,Palm Springs International,"Palm Springs, CA",33.829722,-116.506667,San Francisco International,"San Francisco, CA",37.618889,-122.375556,False
24798426,2019,1,1,11,5,2019-01-11 00:00:00,AS,628VA,1595,34262,...,2023.0,Palm Springs International,"Palm Springs, CA",33.829722,-116.506667,San Francisco International,"San Francisco, CA",37.618889,-122.375556,True
24798428,2019,1,1,11,5,2019-01-11 00:00:00,AS,844VA,1621,30423,...,2023.0,Austin - Bergstrom International,"Austin, TX",30.194444,-97.670000,Seattle/Tacoma International,"Seattle, WA",47.450000,-122.311667,False
24798429,2019,1,1,11,5,2019-01-11 00:00:00,AS,844VA,1621,30423,...,2023.0,Robert Mueller Municipal,"Austin, TX",30.298056,-97.701389,Seattle/Tacoma International,"Seattle, WA",47.450000,-122.311667,False
24798430,2019,1,1,11,5,2019-01-11 00:00:00,AS,839VA,1625,32211,...,2023.0,Harry Reid International,"Las Vegas, NV",36.080000,-115.152222,Portland International,"Portland, OR",45.588611,-122.596944,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32288689,2019,4,12,1,7,2019-12-01 00:00:00,DL,680DA,825,30397,...,2023.0,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,Norman Y. Mineta San Jose International,"San Jose, CA",37.363056,-121.928611,True
32288690,2019,4,12,1,7,2019-12-01 00:00:00,DL,680DA,825,32457,...,2023.0,Norman Y. Mineta San Jose International,"San Jose, CA",37.363056,-121.928611,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,False
32288692,2019,4,12,1,7,2019-12-01 00:00:00,DL,678DL,827,32467,...,2023.0,Fort Lauderdale-Hollywood International,"Fort Lauderdale, FL",26.071667,-80.149722,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,False
32288693,2019,4,12,1,7,2019-12-01 00:00:00,DL,865DA,828,31650,...,2023.0,Minneapolis-St Paul International,"Minneapolis, MN",44.881944,-93.221667,Hartsfield-Jackson Atlanta International,"Atlanta, GA",33.636667,-84.427778,True


In [ ]:
df_top_airline_airport.to_parquet(f"{SCRATCH_DIR}/df_top_airline_airport_2019.parquet")

In [23]:
top_10_airports = (
    pd.concat([df_top_airline_airport["Origin"], df_top_airline_airport["Dest"]])
    .value_counts()
    .head(10)
    .index
)

airport_coords = df[df["Origin"].isin(top_10_airports)]

airport_coords = airport_coords[
    ["Origin", "Origin_LATITUDE", "Origin_LONGITUDE"]
].drop_duplicates(subset=["Origin"])

airport_coords = pd.DataFrame(
    airport_coords.set_index("Origin").reindex(top_10_airports).reset_index()
)

airport_coords["Ranking"] = airport_coords.apply(
    lambda row: f"{row.name + 1}. {row['index']}", axis=1
)

airport_coords

,index,Origin_LATITUDE,Origin_LONGITUDE,Ranking
0,ATL,33.636667,-84.427778,1. ATL
1,LAX,33.942500,-118.408056,2. LAX
2,DFW,32.897222,-97.037778,3. DFW
3,DEN,39.861667,-104.673056,4. DEN
4,ORD,41.976944,-87.908056,5. ORD
5,PHX,33.434167,-112.011667,6. PHX
6,SEA,47.450000,-122.311667,7. SEA
7,SFO,37.618889,-122.375556,8. SFO
8,LAS,36.080000,-115.152222,9. LAS
9,BOS,42.363056,-71.006389,10. BOS


In [24]:
import plotly.express as px

fig = px.scatter_geo(
    airport_coords,
    lat="Origin_LATITUDE",
    lon="Origin_LONGITUDE",
    text="Ranking",
    scope="usa",
    title='Top 10 U.S. Airports by Selected Airlines in 2019 ("WN", "DL", "AA", "B6", "UA", "AS")',
    template="plotly",
    opacity=0.6,
    labels={"Origin_LATITUDE": "Latitude", "Origin_LONGITUDE": "Longitude"},
    size_max=100,
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed